In [1]:
# 1. Hücre
!pip install transformers torch accelerate bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.9 MB/s eta 0:00:00


In [11]:
# 2. Import Libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import warnings

# Suppress unnecessary warnings for clean output
warnings.filterwarnings("ignore")

# --- Model Configuration ---
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

print(f"Loading model: '{model_id}'...")

# 3. Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 4. Load Model (4-bit quantized)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",      # Auto-map to GPU
    torch_dtype=torch.float16,
    load_in_4bit=True       # Enable 4-bit quantization
)

print("Model loaded successfully.")

# 5. Create Text Generation Pipeline
print("Creating chatbot pipeline...")
chatbot = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,  # Max tokens (words) to generate
    temperature=0.7      # Model creativity (0.1=factual, 0.9=creative)
)

print("--- MISTRAL 7B READY! ---")

Loading model: 'mistralai/Mistral-7B-Instruct-v0.2'...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


Model loaded successfully.
Creating chatbot pipeline...
--- MISTRAL 7B READY! ---


In [12]:
# 6. Helper Function for Chatbot
def ask(question):
    print(f"\nYou: {question}")
    print("Just a sec...")

    # Create message list for Mistral-Instruct format
    messages = [{"role": "user", "content": question}]

    # Apply the correct chat template
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # Get the full output from the pipeline
    output = chatbot(prompt)[0]['generated_text']

    # Extract only the answer (text after the [/INST] tag)
    answer = output.split("[/INST]")[-1].strip()

    return answer

# Test 1
question_1 = "Who is Johan Liebert from Naoki Urasawa's Monster?"
answer_1 = ask(question_1)
print(f"Answer: {answer_1}")

# Test 2
question_2 = "Compare Johan Liebert and Judge Holden based on their philosophy"
answer_2 = ask(question_2)
print(f"Answer: {answer_2}")

# Test 3
question_3 = "What is the main idea of 'Science and Art' as a course?"
answer_3 = ask(question_3)
print(f"Mistral: {answer_3}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



You: Who is Johan Liebert from Naoki Urasawa's Monster?
Just a sec...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer: In Naoki Urasawa's manga and anime series "Monster," Johan Liebert is a mysterious and enigmatic character. He is a brilliant surgeon and a renowned pianist. At the beginning of the story, he is depicted as a kind and compassionate person, but as the series progresses, he is revealed to be a manipulative and dangerous individual.

Johan is known for his exceptional intelligence and his ability to manipulate people to achieve his goals. He is also a prime suspect in a series of gruesome murders, which adds to his enigmatic nature. The true nature of Johan and his connection to the murders is a major part of the plot in "Monster."

Despite his dark side, Johan is also portrayed as a tragic figure, as he has a deep longing for human connection and acceptance, but his actions and cold demeanor often push people away. He is a complex character, and his motivations and true intentions are a major source of intrigue and suspense throughout the series.

You: Compare Johan Liebert and J

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer: Johan Liebert and Judge Holden are two complex and enigmatic characters from two literary works: Johan is the main protagonist of Fyodor Dostoevsky's novel "Crime and Punishment," while Judge Holden is a significant character in Cormac McCarthy's "Blood Meridian, or the Evening Redness in the West." Both characters exhibit philosophical leanings that shape their actions and motivations. Here's a comparison of their philosophies:

Johan Liebert:

1. Existentialism: Johan's philosophy can be regarded as existentialist. He is a deeply introspective character who grapples with the meaning of existence and morality. He believes that individuals have the freedom to choose their own path in life, and that their actions define them. This is exemplified by his decision to commit murder, which he justifies by believing that he alone is responsible for his actions and their consequences.

2. Nihilism: Johan's existentialist beliefs also lean towards nihilism. He questions the value of mor